In [ ]:
!pip install -U sentence-transformers

### Датасет

[ссылка](https://www.kaggle.com/datasets/dimka11/cl-cup-it-2023-ds)

### Эмбединги
[ссылка](https://www.kaggle.com/datasets/dimka11/cl-cup-it-2023-embs)


 # Инференс тестового датасета

In [3]:
import numpy as np
import pandas as pd
import scipy
import os
import json
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, ndcg_score
from sentence_transformers import SentenceTransformer

In [4]:
train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)
test = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

### Подготовка датасетов

In [5]:
def prepare_df(train, test=False):
    train = train.explode('comments')
    if not test:
        train['score'] = train['comments'].str['score']
    train['comment'] = train['comments'].str['text']
    train = train.drop(columns=['comments'])
    train['text_id'] = train.index
    return train
train = prepare_df(train)
scores = train['score'].values
test = prepare_df(test, True)
train.head(20)

,text,score,comment,text_id
0,How many summer Y Combinator fundees decided n...,0,Going back to school is not identical with giv...,0
0,How many summer Y Combinator fundees decided n...,1,There will invariably be those who don't see t...,0
0,How many summer Y Combinator fundees decided n...,2,For me school is a way to be connected to what...,0
0,How many summer Y Combinator fundees decided n...,3,I guess it really depends on how hungry you ar...,0
0,How many summer Y Combinator fundees decided n...,4,I know pollground decided to go back to school...,0
1,CBS acquires last.fm for $280m,0,It will be curious to see where this heads in ...,1
1,CBS acquires last.fm for $280m,1,Does this mean that there's now a big-name com...,1
1,CBS acquires last.fm for $280m,2,Also on BBC News: http://news.bbc.co.uk/1/low...,1
1,CBS acquires last.fm for $280m,3,I don't understand what they do that is worth ...,1
1,CBS acquires last.fm for $280m,4,sold out too cheaply. given their leadership p...,1


### Загрузка полученных эмбедингов текстов и комментариев

Для загрузки эмбедингов с диска необходимо прописать пути

get_embs:

* False - загрузка с диска

* True - генерация при помощи модели SentenceTransformer


In [27]:
get_embs = False

if get_embs:
    model_name = 'paraphrase-MiniLM-L6-v2' # Легкая и быстрая модель
    model = SentenceTransformer(model_name)

if get_embs:
    batch_size = 256
    train_text_embs = model.encode(train.groupby('text_id').first()['text'].values, batch_size=batch_size)
    test_text_embs = model.encode(test.groupby('text_id').first()['text'].values, batch_size=batch_size)

    train_comment_embs = model.encode(train['comment'].values, batch_size=batch_size)
    test_comment_embs = model.encode(test['comment'].values, batch_size=batch_size)
    
if get_embs:
    np.save("train_text_embs.npy", train_text_embs)
    np.save("test_text_embs.npy", test_text_embs)
    np.save("train_comment_embs.npy", train_comment_embs)
    np.save("test_comment_embs.npy", test_comment_embs)
else:
    train_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_text_embs.npy' ,allow_pickle=True)
    test_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_text_embs.npy' ,allow_pickle=True)
    train_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_comment_embs.npy' ,allow_pickle=True)
    test_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_comment_embs.npy' ,allow_pickle=True)

Соединяем каждый комментарий с текстом

Это дает + ~0.02 к метрике NDCG

In [28]:
concat_text_comments = True

if (concat_text_comments) and (train_comment_embs.shape[1] < 768):
    train_comment_embs = np.concatenate([np.repeat(train_text_embs, 5, 0), train_comment_embs], axis=1)
    test_comment_embs = np.concatenate([np.repeat(test_text_embs, 5, 0), test_comment_embs], axis=1)

In [31]:
sc = StandardScaler()
X_train = sc.fit_transform(train_comment_embs)
X_test = sc.transform(test_comment_embs)

## Ridge

Предсказание теста

In [69]:
ridge_clf = RidgeClassifier(alpha=0.9)
ridge_clf.fit(X_train, scores)
y_pred = ridge_clf.predict(X_test)
d_t = ridge_clf.decision_function(X_test)
y_pred_probs = scipy.special.softmax(d_t, axis=1)
assert(y_pred_probs.shape[0] == X_test.shape[0])

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.32028e-08): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


Проверим на тренировочных данных

При обучении и валидации моделей точность Ridge классификатора на валидационной и тренировочной выборках была примерно одинакова

In [59]:
y_pred_tr = ridge_clf.predict(X_train)
d = ridge_clf.decision_function(X_train)
y_pred_tr_probs = scipy.special.softmax(d, axis=1)
accuracy_score(scores, y_pred_tr), ndcg_score(np.eye(5)[scores], np.eye(5, dtype=np.int8)[y_pred_tr])

(0.2875980342084057, 0.6346195608469649)

Заменяем классы комментарием после 0-го в соответствии  предсказанными моделью вероятностями.

In [60]:
y_preds_mod = []
for i, item in enumerate(y_pred_tr_probs[::5]):
    arr = y_pred_tr_probs[i*5:i*5+5].copy()
    pred_0 = np.argmax(arr[0])
    arr[:, pred_0] = 0

    pred_1 = np.argmax(arr[1])
    arr[:, pred_1] = 0

    pred_2 = np.argmax(arr[2])
    arr[:, pred_2] = 0

    pred_3 = np.argmax(arr[3])
    arr[:, pred_3] = 0

    pred_4 = np.argmax(arr[4])
    arr[:, pred_4] = 0

    y_preds_mod.extend([pred_0, pred_1, pred_2, pred_3, pred_4])
    
y_preds_mod = np.array(y_preds_mod)

Проверяем точность

In [61]:
accuracy_score(scores, y_preds_mod), ndcg_score(np.eye(5)[scores], np.eye(5, dtype=np.int8)[y_preds_mod])

(0.40869397437207033, 0.6967278787928382)

Делаем тоже самое, с предсказаниями на тестовой выборке

In [70]:
y_preds_mod_test = []
for i, item in enumerate(y_pred[::5]):
    arr = y_pred_probs[i*5:i*5+5].copy()
    pred_0 = np.argmax(arr[0])
    arr[:, pred_0] = 0

    pred_1 = np.argmax(arr[1])
    arr[:, pred_1] = 0

    pred_2 = np.argmax(arr[2])
    arr[:, pred_2] = 0

    pred_3 = np.argmax(arr[3])
    arr[:, pred_3] = 0

    pred_4 = np.argmax(arr[4])
    arr[:, pred_4] = 0

    pred_0, pred_1, pred_2, pred_3, pred_4
    y_preds_mod_test.extend([pred_0, pred_1, pred_2, pred_3, pred_4])
    
y_preds_mod_test = np.array(y_preds_mod_test)

In [71]:
assert (y_pred.shape == y_preds_mod_test.shape)
y_pred = y_preds_mod_test

Записываем y_pred в pandas df и jsonl

In [85]:
submit = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

In [86]:
def write_score(row):
    row_data = row['comments']
    
    row_data[0]['score'] = y_pred[row.name*5]
    row_data[1]['score'] = y_pred[row.name*5+1]
    row_data[2]['score'] = y_pred[row.name*5+2]
    row_data[3]['score'] = y_pred[row.name*5+3]
    row_data[4]['score'] = y_pred[row.name*5+4]
    return row

submit = submit.apply(write_score, axis=1)

In [103]:
submit.iloc[2, 1]

[{'text': 'What I don&#x27;t understand is why drugs developed with public money (government grants and non-profit charities) aren&#x27;t automatically in the public domain.',
  'score': 4},
 {'text': 'To everyone sure this will fail because of the amount of money required, ask yourself this: if you told your mom, and your neighbor, and your mailman, and your boss about the Occulus Rift, or Pebble, or The Peachy Printer, or Zach Braff&#x27;s latest movie, and the opportunity to help make those products a reality by preordering them&#x2F;donating money to them, do you really think any sizable portion of those people would be interested enough to donate?  Of course not.Most successful Kickstarter and IndieGoGo campaigns have an extremely narrow market, the early adopters of technology.  Cancer casts a wider net by several orders of magnitude.  There&#x27;s no reason to think that, with a good viral marketing campaign, something like this couldn&#x27;t raise several orders of magnitude mo

In [87]:
submit.to_json('submit.json', lines=True, orient='records')

Данный подход дает 
NDCG ~ 0.648
Accuracy ~ 0.28
Accuracy 0-го класса ~ 0.6


(0.40869397437207033, 0.6967278787928382)

Блокноты с обучением и валидацей других моделей доступны в репозитории

Результаты других моделей.



Выводы.